**Training toxicity classifier model**

In [ ]:
! pip install -q kaggle

Upload kaggle.json

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Download dataset

In [ ]:
! kaggle competitions download -c 'jigsaw-toxic-comment-classification-challenge'
! unzip 'jigsaw-toxic-comment-classification-challenge.zip'

 99% 52.0M/52.6M [00:04<00:00, 15.6MB/s]
100% 52.6M/52.6M [00:04<00:00, 12.9MB/s]
Archive:  jigsaw-toxic-comment-classification-challenge.zip
  inflating: sample_submission.csv.zip  
  inflating: test.csv.zip            
  inflating: test_labels.csv.zip     
  inflating: train.csv.zip           


In [ ]:
! unzip 'train.csv.zip'
! unzip 'test.csv.zip'
! unzip 'test_labels.csv.zip'

Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  test_labels.csv.zip
  inflating: test_labels.csv         


Download pretrained glove word embedding

In [ ]:
!wget https://nlp.stanford.edu/data/glove.twitter.27B.zip
!unzip -q glove.twitter.27B.zip

## below is temporary to get embedding from my gdrive
# from google.colab import drive
# drive.mount('/content/gdrive')
# ! cp gdrive/MyDrive/glove.twitter.27B.200d.txt glove.twitter.27B.200d.txt 

--2023-06-03 12:06:24--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2023-06-03 12:06:25--  https://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [application/zip]
Saving to: ‘glove.twitter.27B.zip’

glove.twitter.27B.z 100%[===================>]   1.42G  5.02MB/s    in 4m 44s  

2023-06-03 12:11:10 (5.11 MB/s) - ‘glove.twitter.27B.zip’ saved [1520408563/1520408563]



In [ ]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization, Embedding
from tensorflow.keras import layers

Load and preprocess dataset

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_test_labels = pd.read_csv('test_labels.csv')

## combine df_test and its labels then throw away rows with -1 values
df_test_labels_normalized = df_test_labels[df_test_labels['toxic']!=-1]
df_test_normalized = df_test.set_index('id').join(df_test_labels_normalized.set_index('id'), how='right')

In [ ]:
feature = ['comment_text']
target = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

## convert into tf.data.Dataset
train_data = tf.data.Dataset.from_tensor_slices((df_train[feature], df_train[target]))
test_data = tf.data.Dataset.from_tensor_slices((
    df_test_normalized[feature],
    df_test_normalized[target]
))

Use TextVectorization to convert text to sequences

In [ ]:
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200, ## max_tokens denotes number of words to be tokenized
                               pad_to_max_tokens=True)

vectorizer.adapt(train_data.map(lambda x, y: x).batch(2000)) ## use .map() to get the input only since in the dataset there are input and label

voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

Load pretrained word embedding and put it into dictionary

In [ ]:
path_to_glove_file = "glove.twitter.27B.200d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
  for line in f:
    word, coefs = line.split(maxsplit=1)
    coefs = np.fromstring(coefs, "f", sep=" ")
    embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 1193514 word vectors.


Only use words from pretrained word embedding that exist in vectorizer.get_vocabulary()

In [ ]:
num_tokens = len(voc) + 2  ## TextVectorization already includes OOV and padding, but pretrained glove file also includes OOV and padding so we add 2
embedding_dim = 200  ## 200 as dimension comes from pretrained word embedding
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    # Words not found in embedding index will be all-zeros.
    # This includes the representation for "padding" and "OOV"
    embedding_matrix[i] = embedding_vector
    hits += 1
  else:
    misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 17856 words (2144 misses)


Initialize embedding layer with 20000 of pretrained embedding vectors

In [ ]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

Create sequential model

In [ ]:
model = tf.keras.Sequential([
    layers.Input(shape=(200,), dtype='int64'),
    embedding_layer,
    layers.Bidirectional(layers.GRU(128, return_sequences=False)),
    layers.Dense(128, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(6, activation='sigmoid')
])

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 200)          4000400   
                                                                 
 bidirectional_7 (Bidirectio  (None, 256)              253440    
 nal)                                                            
                                                                 
 dense_14 (Dense)            (None, 128)               32896     
                                                                 
 dense_15 (Dense)            (None, 256)               33024     
                                                                 
 dense_16 (Dense)            (None, 6)                 1542      
                                                                 
Total params: 4,321,302
Trainable params: 320,902
Non-trainable params: 4,000,400
______________________________________

Transform train data and test data from raw text to sequences

In [ ]:
## define function to convert raw text to sequences
def to_sequence(x, y):
  return vectorizer(x), y

## batch, cache, and prefetch
## set batch of 512, since we have big vram and so we can better utilize the GPU
## batch size tradeoff:
## - big batch -> better GPU utilization -> faster training time -> lower accuracy
## - small batch -> worse GPU utilization -> slower training time -> higher accuracy
train_data = train_data.batch(512).map(to_sequence).cache().prefetch(tf.data.AUTOTUNE) 
test_data = test_data.batch(512).map(to_sequence).cache().prefetch(tf.data.AUTOTUNE)  

Compile and train model

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), 
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006),
              metrics=[tf.keras.metrics.Recall(thresholds=0.5),     ## assume the threshold is 0.5
                       tf.keras.metrics.Precision(thresholds=0.5)])  ## although it's incorrect, but can give
                                                                    ## depiction of training performance
history = model.fit(train_data,
                    epochs=8, 
                    validation_data=test_data,
                    verbose=1)


Epoch 1/8
312/312 [==============================] - 31s 86ms/step - loss: 0.0911 - recall_4756: 0.4617 - precision_4756: 0.6671 - val_loss: 0.0727 - val_recall_4756: 0.6166 - val_precision_4756: 0.6267
Epoch 2/8
312/312 [==============================] - 28s 88ms/step - loss: 0.0487 - recall_4756: 0.6608 - precision_4756: 0.8172 - val_loss: 0.0687 - val_recall_4756: 0.6352 - val_precision_4756: 0.6415
Epoch 3/8
312/312 [==============================] - 26s 82ms/step - loss: 0.0451 - recall_4756: 0.6845 - precision_4756: 0.8250 - val_loss: 0.0655 - val_recall_4756: 0.6541 - val_precision_4756: 0.6559
Epoch 4/8
312/312 [==============================] - 26s 82ms/step - loss: 0.0427 - recall_4756: 0.7023 - precision_4756: 0.8326 - val_loss: 0.0640 - val_recall_4756: 0.6643 - val_precision_4756: 0.6583
Epoch 5/8
312/312 [==============================] - 25s 82ms/step - loss: 0.0409 - recall_4756: 0.7149 - precision_4756: 0.8378 - val_loss: 0.0638 - val_recall_4756: 0.6735 - val_precisio

Use greedy search to find out the best threshold that yields the best f1 score for each label

In [ ]:
## make prediction to all train and test dataset
train_prediction = model.predict(train_data.map(lambda x, y: x))
test_prediction = model.predict(test_data.map(lambda x, y: x))

f1_train_scores = []
f1_test_scores = []
thresholds = []

## iterate all 6 labels
for idx, tgt in enumerate(target):
  best_threshold = 0
  max_f1_train = 0
  max_f1_test = 0

  ## check threshold from 0.01 to 0.99, with 0.01 step
  for threshold in np.arange(0.01, 1, 0.01): 
    train_precision = tf.keras.metrics.Precision(thresholds=threshold)
    train_recall = tf.keras.metrics.Recall(thresholds=threshold)
    test_precision = tf.keras.metrics.Precision(thresholds=threshold)
    test_recall = tf.keras.metrics.Recall(thresholds=threshold)

    ## [:, idx] to take slice of 1 label only
    ## use data from pandas dataframe since it's not batched
    train_precision.update_state(df_train[tgt], train_prediction[:,idx])
    train_recall.update_state(df_train[tgt], train_prediction[:,idx])
    test_precision.update_state(df_test_normalized[tgt], test_prediction[:,idx])
    test_recall.update_state(df_test_normalized[tgt], test_prediction[:,idx])

    train_precision = train_precision.result().numpy()
    train_recall = train_recall.result().numpy()
    test_precision = test_precision.result().numpy()
    test_recall = test_recall.result().numpy()

    ## get f1 score from train and test data
    f1_train = 2 * train_precision * train_recall / (train_precision + train_recall)
    f1_test = 2 * test_precision * test_recall / (test_precision + test_recall)

    ## get the best f1 score based on test data
    if f1_test > max_f1_test:
      max_f1_train = f1_train
      max_f1_test = f1_test
      best_threshold = threshold

  print('label: ', tgt)
  print('f1 train: ', max_f1_train)
  print('f1 test: ', max_f1_test)
  print('threshold: ', best_threshold, '\n')

  ## store f1 score of train and test data, also store best threshold
  f1_train_scores.append(max_f1_train)
  f1_test_scores.append(max_f1_test)
  thresholds.append(best_threshold)

print('average f1 train: ', np.mean(f1_train_scores))
print('average f1 test: ', np.mean(f1_test_scores))

125/125 [==============================] - 3s 26ms/step
label:  toxic
f1 train:  0.7922951538385359
f1 test:  0.7071478879771347
threshold:  0.67 

label:  severe_toxic
f1 train:  0.5312169695572128
f1 test:  0.42505590467331844
threshold:  0.31 

label:  obscene
f1 train:  0.8391805388562931
f1 test:  0.7001892360513065
threshold:  0.47000000000000003 



<ipython-input-47-73d37a6713ad>:36: RuntimeWarning: invalid value encountered in double_scalars
  f1_test = 2 * test_precision * test_recall / (test_precision + test_recall)
<ipython-input-47-73d37a6713ad>:35: RuntimeWarning: invalid value encountered in double_scalars
  f1_train = 2 * train_precision * train_recall / (train_precision + train_recall)


label:  threat
f1 train:  0.5629053291459745
f1 test:  0.5063938579786176
threshold:  0.4 

label:  insult
f1 train:  0.7903103093571139
f1 test:  0.6743336770115216
threshold:  0.25 

label:  identity_hate
f1 train:  0.6292880712247115
f1 test:  0.5831621893747855
threshold:  0.27 

average f1 train:  0.6908660619966404
average f1 test:  0.5993804588444472


Try predicting with the model

In [ ]:
np.set_printoptions(precision=8, suppress=True)  ## print in decimal number, not scientific
seq = vectorizer([["i will smack your face"]]) ## turn text into sequence first using vectorizer
prediction = model.predict(seq)
print(target)   ## print all label names
print(prediction > thresholds)    ## print prediction values as true or false according to thresholds
print(prediction)   ## print prediction value

1/1 [==============================] - 0s 24ms/step
['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
[[ True False False  True False False]]
[[0.9833372  0.04298376 0.18612957 0.62194926 0.04752325 0.00073487]]


Append TextVectorization layer to the model, so we don't need to do separate preprocessing and can directly input raw text to the model

In [ ]:
# Start by creating an explicit input layer. It needs to have a shape of  
# (1,) (because we need to guarantee that there is exactly one string  
# input per batch), and the dtype needs to be 'string'.
end_to_end_model = tf.keras.Sequential([
    tf.keras.Input(shape=(1,), dtype=tf.string),
    vectorizer,
    model
])

end_to_end_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 200)              0         
 torization)                                                     
                                                                 
 sequential_4 (Sequential)   (None, 6)                 4321302   
                                                                 
Total params: 4,321,302
Trainable params: 320,902
Non-trainable params: 4,000,400
_________________________________________________________________


Try the end-to-end model

In [ ]:
end_to_end_model.predict([["i will smack your face"]])  ## can directly input raw text, no need vectorizer

1/1 [==============================] - 2s 2s/step


array([[0.9833372 , 0.04298376, 0.18612957, 0.62194926, 0.04752325,
        0.00073487]], dtype=float32)

Save the model

In [ ]:
end_to_end_model.save('./saved_model/end-to-end')  ## model with TextVectorization layer

Save to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
! cp -r saved_model gdrive/MyDrive

Try to load the saved models

In [ ]:
loaded_end_to_end_model = tf.keras.models.load_model('saved_model/end-to-end')

Make prediction with loaded models

In [ ]:
print('loaded_end_to_end_model: ', loaded_end_to_end_model.predict( [["i will smack your face"]] ))

1/1 [==============================] - 1s 685ms/step
loaded_end_to_end_model:  [[0.9833372  0.04298376 0.18612957 0.62194926 0.04752325 0.00073487]]
